<h1><center>Segmenting and Clustering Neighbourhoods in Toronto</center></h1>
<h3><center>Federico Sciuca</center></h3>

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Data Scraping</a>

2. <a href="#item2">Get Latitude and Logitude of each Neighbourhood</a>

3. <a href="#item3">Analyze Each Neighbourhood</a>

4. <a href="#item4">Cluster Neighbourhoods using K-Means</a>

5. <a href="#item5">Examine Clusters</a>
</font>
</div>

<a id='item1'></a>

<h2><center id="#item1">Part 1 - Data scraping from Wikipedia</center></h2>

Building the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

<img src="https://d3c33hcgiwev3.cloudfront.net/imageAssetProxy.v1/7JXaz3NNEeiMwApe4i-fLg_40e690ae0e927abda2d4bde7d94ed133_Screen-Shot-2018-06-18-at-7.17.57-PM.png?expiry=1574121600000&hmac=CV78Cmf2C1BDNgZClNp2AYTXMbI6rD7oW2EfZGsxT8Y">

<p>To create the above dataframe:</p>
<ul>
    <li>The dataframe will consist of three columns: PostalCode, Borough, and Neighbourhood</li>
    <li>Only process the cells that have an assigned borough. Ignore cells with a borough that is <strong>Not assigned.</strong></li>
    <li>More than one neighbourhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that <strong>M5A</strong> is listed twice and has two neighbourhoods: <strong>Harbourfront </strong>and <strong>Regent Park</strong>. These two rows will be combined into one row with the neighbourhoods separated with a comma as shown in <strong>row 11 </strong> in the above table.</li>
</ul>

### First of all, I install the libraries I need to scrape the Wikipedia Table

In [1]:
# Get BeautifulSoup Library to scrape the web page and 
!conda install -c anaconda beautifulsoup4 -y

# Get lxlm
!conda install -c anaconda lxml -y

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.6.14
  latest version: 4.8.0rc0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /srv/conda/envs/notebook

  added / updated specs:
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.8.1       |           py37_0         153 KB  anaconda
    ca-certificates-2019.10.16 |                0         131 KB  anaconda
    certifi-2019.9.11          |           py37_0         154 KB  anaconda
    openssl-1.1.1              |       h7b6447c_0         5.0 MB  anaconda
    soupsieve-1.9.4            |           py37_0          61 KB  anaconda
    ------------------------------------------------------------
                                           Total:         5.5 MB

The following NEW pa

In [2]:
# Install requests library
!conda install -c anaconda requests -y

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.6.14
  latest version: 4.8.0rc0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /srv/conda/envs/notebook

  added / updated specs:
    - requests


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    asn1crypto-1.2.0           |           py37_0         162 KB  anaconda
    cffi-1.13.2                |   py37h2e261b9_0         225 KB  anaconda
    chardet-3.0.4              |        py37_1003         173 KB  anaconda
    cryptography-2.8           |   py37h1ba5d50_0         612 KB  anaconda
    idna-2.8                   |           py37_0         101 KB  anaconda
    pycparser-2.19             |             py_0          89 KB  anaconda
    pyopenssl-19.0.0           |           py37_0          82 KB  anaconda
    pys

#### The second step consist in importing the libraries we need to implement the data analysis

In [3]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib as plt
from bs4 import BeautifulSoup
import requests
import lxml

### Let's start!  
  
##### We can start defining the source we need to scrape and print the Html code

In [4]:
# Define the link for the webpage you need to scrape
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

# Make the soup
soup = BeautifulSoup(source, 'lxml')

# Print the soup - I commented the code to make the Notebook easy to read
# print(soup.prettify())

From the Html code we need to identify and save in a new variable the code referred to the table with the data we need

In [5]:
# Let's find the table and get the entire tbody
table = soup.find("table", class_="wikitable sortable").tbody

# Print the table in Html code
print(table.prettify())

<tbody>
 <tr>
  <th>
   Postcode
  </th>
  <th>
   Borough
  </th>
  <th>
   Neighbourhood
  </th>
 </tr>
 <tr>
  <td>
   M1A
  </td>
  <td>
   Not assigned
  </td>
  <td>
   Not assigned
  </td>
 </tr>
 <tr>
  <td>
   M2A
  </td>
  <td>
   Not assigned
  </td>
  <td>
   Not assigned
  </td>
 </tr>
 <tr>
  <td>
   M3A
  </td>
  <td>
   <a href="/wiki/North_York" title="North York">
    North York
   </a>
  </td>
  <td>
   <a href="/wiki/Parkwoods" title="Parkwoods">
    Parkwoods
   </a>
  </td>
 </tr>
 <tr>
  <td>
   M4A
  </td>
  <td>
   <a href="/wiki/North_York" title="North York">
    North York
   </a>
  </td>
  <td>
   <a href="/wiki/Victoria_Village" title="Victoria Village">
    Victoria Village
   </a>
  </td>
 </tr>
 <tr>
  <td>
   M5A
  </td>
  <td>
   <a href="/wiki/Downtown_Toronto" title="Downtown Toronto">
    Downtown Toronto
   </a>
  </td>
  <td>
   <a href="/wiki/Regent_Park" title="Regent Park">
    Harbourfront
   </a>
  </td>
 </tr>
 <tr>
  <td>
   M6A
  </td>
  

We can note that every single row is defined between the tr tag. Let's find the rows

In [6]:
# Define the rows
rows = table.find_all("tr")

The columns names are definded between the tag th

In [7]:
# Identify the columns name
columns = [v.text.replace("\n", "") for v in rows[0].find_all("th")]
print(columns)

['Postcode', 'Borough', 'Neighbourhood']


Now it's time to create a new Pandas DataFrame and assign to the columns the columns name we have just extract

In [8]:
# Define the DataFrame
df = pd.DataFrame(columns=columns)

# Check if the DataFrame has been created correctly
df

,Postcode,Borough,Neighbourhood


The dataframe will consist of three columns: PostalCode, Borough, and Neighbourhood.  
Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [9]:
for i in range(1, len(rows)):
    # Identify the cells for each row
    tds = rows[i].find_all("td")
    
    # If the "Borough" is missing, go to the next row, otherwise assign to the variable values the value of each cell
    if tds[1].text != "Not assigned":
        values = [tds[0].text.replace("\n", ""), 
                  tds[1].text.replace("\n", ""), 
                  tds[2].text.replace("\n", "")]
    else:
        continue
    
    # Append the new values to the DataFrame
    df = df.append(pd.Series(values, index=columns), ignore_index=True)
    df.head(15)

In [10]:
df.head(15)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Queen's Park,Not assigned
6,M9A,Queen's Park,Queen's Park
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


Check how many rows and columns we have:

In [11]:
df.shape

(210, 3)

Let's check how many unique values each column has

In [12]:
df.T.apply(lambda x: x.nunique(), axis=1)

Postcode         103
Borough           11
Neighbourhood    208
dtype: int64

More than one neighbourhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighbourhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighbourhoods separated with a comma as shown in row 11 in the above table.  
  
If a cell has a borough but a Not assigned neighbourhood, then the neighbourhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighbourhood columns will be Queen's Park.

In [13]:
# define a for loop that replace the "Not assigned" values in the "Neighbourhood" column with the value of the "Borough" from the same row
for i in range(0, len(df)):
    
    if df.iloc[i]["Neighbourhood"] == 'Not assigned':
        df.iloc[i]["Neighbourhood"] = df.iloc[i]["Borough"]
    else:
        pass

In [14]:
df.head(15)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Queen's Park,Queen's Park
6,M9A,Queen's Park,Queen's Park
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


Now we need to group the DataFrame by the "Postcode" and the "Borough" aggregating the "Neighbourhood" as a comma separated format

In [15]:
df_grouped = df.groupby(["Postcode", "Borough"])["Neighbourhood"].agg([('Neighbourhood', ', '.join)]).reset_index()
df_grouped

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


Let's check how many rows and columns we have

In [16]:
df_grouped.shape

(103, 3)

In [17]:
df_grouped.to_csv("TorontoPostalData.csv")

---

<a id='item2'></a>

<h2><center>Part 2 - Get Latitude and Longitude</center></h2>

### Import necessary Libraries

In [18]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.6.14
  latest version: 4.8.0rc0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /srv/conda/envs/notebook

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    certifi-2019.9.11          |           py37_0         147 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following 

##### Now that we have built a dataframe of the postal code of each neighbourhood along with the borough name and neighbourhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighbourhood.  
  
##### In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighbourhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.  
  
##### The problem with this Package is we have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So we can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that we get the coordinates for all of our neighbourhood, we can run a while loop for each postal code. Taking postal code M5G as an example, our code would look something like this:

The code above takes a lot of time to get the latitude and longitude for each neighbourhood. For this reason I'll use the csv file provided.

In [19]:
df_geo = pd.read_csv("http://cocl.us/Geospatial_data")

# Rename the Postal Code column
df_geo.rename(columns = {"Postal Code" : "Postcode"}, inplace =True)

df_geo.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [20]:
df_grouped.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [21]:
# Because the Postcode is unique for each row, let's try to join the tables:
df_merged = df_grouped.join(df_geo.set_index("Postcode"), on="Postcode")

# df_merged = pd.merge( df_grouped, df_geo, left_on = "Postcode", right_on = "Postal Code")
# df_merged.drop(columns=["Postal Code"], inplace=True)
df_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [22]:
df_merged.shape

(103, 5)

In [23]:
df_merged.reset_index()
df_merged

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [24]:
#missing = ["Upper Rouge", "Newtonbrook, Willowdale", "Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park"]

#pd.set_option('display.max_rows', 105)
#df_merged[["Neighbourhood"]]

#df_merged["Neighbourhood"].where(df_merged["Neighbourhood"] == missing[2])

print(df_merged.iloc[16])
print("------------------")
print(df_merged.iloc[21])
print("------------------")
#print(df_merged.iloc[94])

Postcode                 M1X
Borough          Scarborough
Neighbourhood    Upper Rouge
Latitude             43.8361
Longitude           -79.2056
Name: 16, dtype: object
------------------
Postcode                             M2M
Borough                       North York
Neighbourhood    Newtonbrook, Willowdale
Latitude                         43.7891
Longitude                       -79.4085
Name: 21, dtype: object
------------------
Postcode                                                       M9B
Borough                                                  Etobicoke
Neighbourhood    Cloverdale, Islington, Martin Grove, Princess ...
Latitude                                                   43.6509
Longitude                                                 -79.5547
Name: 94, dtype: object


---

<a id='item3'></a>

<h2><center>Part 3 - Analyze Each neighbourhood in Toronto</center></h2>

Next, we are going to start utilizing the Foursquare API to explore the neighbourhoods and segment them.  
  
#### Define Foursquare Credentials and Version


In [34]:
CLIENT_ID = 'B5KWWDFHTON23TDXFR2TXATYBJQMYXICEEJWOSTX2YIAUZIC' # your Foursquare ID
CLIENT_SECRET = 'FVZXWADC3JUKZKIMRC40LFNTRSHC51MDTVKWHF4IJMGXTDC5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: B5KWWDFHTON23TDXFR2TXATYBJQMYXICEEJWOSTX2YIAUZIC
CLIENT_SECRET:FVZXWADC3JUKZKIMRC40LFNTRSHC51MDTVKWHF4IJMGXTDC5


Now, we go to use the function we wrote in another notebook to get the top 100 venues that are in each Neighbor within a radius of 500 meters.

In [35]:
def getNearbyVenues(names, latitudes, longitudes, radius = 500, LIMIT = 100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

I'll use the function definded in another Notebook to run the above function on each neighbourhood and create a new dataframe called *df_toronto_venues*.

In [44]:
df_toronto_venues = getNearbyVenues(names = df_merged['Neighbourhood'],
                                   latitudes = df_merged['Latitude'],
                                   longitudes = df_merged['Longitude'],
                                  )
print("----------------------------------------------------")
print("Done!")

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

Let's check the first 10 rows of the new DataFrame and the shape of it.

In [45]:
df_toronto_venues.head(10)

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
5,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
6,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location
7,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Woburn Medical Centre,43.766631,-79.192286,Medical Center
8,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Lawrence Ave E & Kingston Rd,43.767704,-79.189490,Intersection
9,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Eggsmart,43.767800,-79.190466,Breakfast Spot


In [46]:
df_toronto_venues.shape

(2245, 7)

In [47]:
df_toronto_venues.groupby("Neighbourhood").count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",3,3,3,3,3,3
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",9,9,9,9,9,9
"Alderwood, Long Branch",8,8,8,8,8,8
"Bathurst Manor, Downsview North, Wilson Heights",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
Berczy Park,57,57,57,57,57,57


In [48]:
print("There are " , len(df_toronto_venues["Venue Category"].unique()), " unique categories in the dataset")

There are  269  unique categories in the dataset


Let's find the missing Neighbourhood

In [49]:
df_merged_neigh = df_merged["Neighbourhood"].tolist()
df_t_venues_neigh = df_toronto_venues["Neighbourhood"].unique().tolist()

In [50]:
for n in df_merged_neigh:
    
    if n in df_t_venues_neigh:
        pass
    else:
        print(n, " ----------------------------- MISSING NEIGHBOURHOOD!")

Upper Rouge  ----------------------------- MISSING NEIGHBOURHOOD!
Newtonbrook, Willowdale  ----------------------------- MISSING NEIGHBOURHOOD!


---

 Now it's time to create dummy variables for categorical features.

In [51]:
# one hot encoding
df_toronto_dummies = pd.get_dummies(df_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
df_toronto_dummies['Neighbourhood'] = df_toronto_venues['Neighbourhood'] 

# move neighbourhood column to the first column
temp_neigh = df_toronto_dummies['Neighbourhood']
df_toronto_dummies.drop(labels=['Neighbourhood'], axis=1,inplace = True)
df_toronto_dummies.insert(0, 'Neighbourhood', temp_neigh)

df_toronto_dummies.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
df_toronto_dummies.shape

(2245, 270)

Now we need to group rows by neighbourhood and take the mean of the frequency of occurrence of each category

In [53]:
df_toronto_dcategories = df_toronto_dummies.groupby("Neighbourhood").mean().reset_index()
df_toronto_dcategories.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03,...,0.02,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
4,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0


In [54]:
df_toronto_dcategories.shape

(100, 270)

Now let's create a dataframe with the top 10 venues for each neighbourhood

In [55]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [56]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = df_toronto_dcategories['Neighbourhood']

for ind in np.arange(df_toronto_dcategories.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_toronto_dcategories.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Bar,Thai Restaurant,Breakfast Spot,Cosmetics Shop,Hotel,Restaurant,American Restaurant
1,Agincourt,Lounge,Latin American Restaurant,Breakfast Spot,Skating Rink,Ethiopian Restaurant,Event Space,Falafel Restaurant,Empanada Restaurant,Diner,Electronics Store
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Playground,Park,Coffee Shop,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Fried Chicken Joint,Sandwich Place,Discount Store,Pizza Place,Beer Store,Japanese Restaurant,Fast Food Restaurant,Pharmacy,Empanada Restaurant
4,"Alderwood, Long Branch",Pizza Place,Skating Rink,Coffee Shop,Pub,Sandwich Place,Gym,Pharmacy,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


In [57]:
neighbourhoods_venues_sorted.shape

(100, 11)

---

<a id='item4'></a>

<h2><center>Part 4 - Clustering using K-means</center></h2>

First of all we need to import the libraries. In this case I need to import the KMean function from SciKit-learn.

In [66]:
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Than, we need to drop the Neighbourhood column from the DataFrame df_toronto_categories

In [67]:
df_toronto_dcategories_nn = df_toronto_dcategories.drop(columns={"Neighbourhood"})
df_toronto_dcategories_nn

,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0.000000,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.0000,0.030000,0.000000,0.00,0.000000,0.000000,0.030000,0.000000,0.000000,0.000000,0.000000,0.0

After that, let's define and train the KMean algorithm to identify 5 different clusters.

In [68]:
k_clusters = 5

k_clusters_fit = KMeans(k_clusters, random_state = 4).fit(df_toronto_dcategories_nn)

In [69]:
k_clusters_fit.labels_

array([3, 3, 0, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 0, 3, 3, 3, 3, 3,
       3, 3, 2, 3, 4, 3, 3, 3, 3, 2, 3, 3, 3, 3, 1, 3, 3, 0, 3, 0, 1, 3,
       3, 3, 0, 2, 3, 3, 3, 3, 3, 3, 2, 3, 1, 3, 3, 2, 3, 2, 3, 0, 3, 3,
       3, 0, 3, 3, 3, 3, 0, 3, 0, 1, 2, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3,
       2, 0, 3, 3, 3, 0, 3, 3, 3, 2, 3, 0], dtype=int32)

Now we need to create a DataFrame that contains everything we need to examine the clusters.  
We are going to marge the cluster labels with the original DataFrame and the top 10 venues for each Neighborhood.

In [70]:
# add clustering labels
# df.insert(loc, column_name, values)
neighbourhoods_venues_sorted.insert(0, "Cluster Label", k_clusters_fit.labels_)

ValueError: cannot insert Cluster Label, already exists

In [72]:
# Create a copy of the DataFrame
df_final = df_merged

# Join the DataFrame
df_final = df_final.join(neighbourhoods_venues_sorted.set_index("Neighbourhood"), on="Neighbourhood")

# For some reason 3 data had been lost after using getNearbyVenues at cell 26. I could try to extract them from the original dataset and analize them separately but for now I just drop the nan values
df_final.dropna(inplace=True)

#df_final["Cluster Label"].isnull()

# Convert Cluster Label in integers
df_final["Cluster Label"] = df_final["Cluster Label"].astype(int)

df_final.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,2,Fast Food Restaurant,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio,Dessert Shop
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,3,Bar,History Museum,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,3,Pizza Place,Medical Center,Intersection,Rental Car Location,Mexican Restaurant,Breakfast Spot,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore
3,M1G,Scarborough,Woburn,43.770992,-79.216917,3,Coffee Shop,Korean Restaurant,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Yoga Studio
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,3,Bakery,Bank,Hakka Restaurant,Caribbean Restaurant,Thai Restaurant,Athletics & Sports,Fried Chicken Joint,Donut Shop,Dog Run,Doner Restaurant


In [75]:
# Toronto latitude and longitude
latitude = 43.651070
longitude = -79.347015

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(k_clusters)
ys = [i + x + (i*x)**2 for i in range(k_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_final['Latitude'], df_final['Longitude'], df_final['Neighbourhood'], df_final['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

---

<a id='item5'></a>

<h2><center>Part 5 - Examine Clusters</center></h2>

### Cluster 1

In [77]:
df_final.loc[df_final['Cluster Label'] == 0, df_final.columns[[1] + list(range(5, df_final.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,0,Playground,Park,Coffee Shop,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
20,North York,0,Park,Cafeteria,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
23,North York,0,Bank,Park,Convenience Store,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
25,North York,0,Food & Drink Shop,Fireworks Store,Park,Yoga Studio,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
30,North York,0,Airport,Park,Electronics Store,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
40,East York,0,Park,Coffee Shop,Convenience Store,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
44,Central Toronto,0,Bus Line,Park,Swim School,Yoga Studio,Drugstore,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
48,Central Toronto,0,Tennis Court,Trail,Playground,Park,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
50,Downtown Toronto,0,Park,Trail,Playground,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
64,Central Toronto,0,Jewelry Store,Park,Sushi Restaurant,Trail,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop


### Cluster 2

In [81]:
df_final.loc[df_final['Cluster Label'] == 1, df_final.columns[[1] + list(range(5, df_final.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,North York,1,Home Service,Baseball Field,Food Truck,Dim Sum Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio
63,Central Toronto,1,Home Service,Garden,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio
91,Etobicoke,1,Home Service,Baseball Field,Dim Sum Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Yoga Studio
97,North York,1,Baseball Field,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Field


### Cluster 3

In [82]:
df_final.loc[df_final['Cluster Label'] == 2, df_final.columns[[1] + list(range(5, df_final.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,2,Fast Food Restaurant,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio,Dessert Shop
13,Scarborough,2,Pizza Place,Pharmacy,Rental Car Location,Italian Restaurant,Fast Food Restaurant,Fried Chicken Joint,Convenience Store,Noodle House,Thai Restaurant,Bank
15,Scarborough,2,Grocery Store,Chinese Restaurant,Fast Food Restaurant,Pharmacy,Coffee Shop,Sandwich Place,Bubble Tea Shop,Breakfast Spot,Pizza Place,Eastern European Restaurant
17,North York,2,Fast Food Restaurant,Pool,Golf Course,Mediterranean Restaurant,Dog Run,Yoga Studio,Drugstore,Diner,Discount Store,Doner Restaurant
35,East York,2,Pizza Place,Fast Food Restaurant,Gym / Fitness Center,Gastropub,Bus Line,Pet Store,Intersection,Bank,Athletics & Sports,Pharmacy
72,North York,2,Pizza Place,Park,Japanese Restaurant,Pub,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
80,York,2,Fast Food Restaurant,Restaurant,Sandwich Place,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
81,York,2,Pizza Place,Grocery Store,Bus Line,Convenience Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
100,Etobicoke,2,Pizza Place,Park,Bus Line,Mobile Phone Shop,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Yoga Studio
101,Etobicoke,2,Grocery Store,Fried Chicken Joint,Sandwich Place,Discount Store,Pizza Place,Beer Store,Japanese Restaurant,Fast Food Restaurant,Pharmacy,Empanada Restaurant


### Cluster 4

In [83]:
df_final.loc[df_final['Cluster Label'] == 3, df_final.columns[[1] + list(range(5, df_final.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,3,Bar,History Museum,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
2,Scarborough,3,Pizza Place,Medical Center,Intersection,Rental Car Location,Mexican Restaurant,Breakfast Spot,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore
3,Scarborough,3,Coffee Shop,Korean Restaurant,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Yoga Studio
4,Scarborough,3,Bakery,Bank,Hakka Restaurant,Caribbean Restaurant,Thai Restaurant,Athletics & Sports,Fried Chicken Joint,Donut Shop,Dog Run,Doner Restaurant
5,Scarborough,3,Playground,Grocery Store,Cosmetics Shop,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
6,Scarborough,3,Discount Store,Department Store,Bus Station,Coffee Shop,Yoga Studio,Drugstore,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
7,Scarborough,3,Bus Line,Bakery,Soccer Field,Intersection,Bus Station,Metro Station,Park,Fast Food Restaurant,Electronics Store,Eastern European Restaurant
8,Scarborough,3,Motel,American Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio,Dim Sum Restaurant
9,Scarborough,3,College Stadium,Skating Rink,General Entertainment,Café,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
10,Scarborough,3,Indian Restaurant,Vietnamese Restaurant,Thrift / Vintage Store,Chinese Restaurant,Pet Store,Electronics Store,Empanada Restaurant,Eastern European Restaurant,Dumpling Restaurant,Drugstore


### Cluster 5

In [84]:
df_final.loc[df_final['Cluster Label'] == 4, df_final.columns[[1] + list(range(5, df_final.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
94,Etobicoke,4,Golf Course,Yoga Studio,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant


---

<h1><center>Thank you!</center></h1>
<h4><center><a href="https://www.linkedin.com/in/federico-sciuca/" target="_blank">Federico Sciuca</a></center></h4>